# Recognize text from slides

## Download slides images from a random website just for testing

In [2]:
import requests
try:
    from bs4 import BeautifulSoup
except:
    %pip install beautifulsoup4
    from bs4 import BeautifulSoup

import os


URL_base = "https://www.slideteam.net/in-powerpoint/market-segmentation-dataset-ppt-presentation-templates-and-google-slides"

cwd = os.getcwd()
if not os.path.isdir(os.path.join(cwd,"slides")):
    os.mkdir(os.path.join(cwd,"slides"))

text = requests.get(URL_base).text
soup = BeautifulSoup(text, "html.parser")

def is_pres_sample_link(link):
    return (".html" in link['href'].split("/")[-1] 
                and "ww.slideteam.net" in link['href'].split("/")[-2]
                and not "one-page-summary" in link['href'].split("/")[-1]
                and not "powerpoint-images" in link['href'].split("/")[-1]
                and not "company-profiles.html" in link['href'].split("/")[-1])

links = set()
for link in soup.find_all('a', href=True):
    if is_pres_sample_link(link):
        links.add(link['href'])
counter = 0
imgs_set = set()        # prevent saving different size pictures from different links
for link in links:
    text = requests.get(link).text
    soup = BeautifulSoup(text, "html.parser")
    for item in soup.find_all('img'):
        try:
            img_url = item['src'].replace("330x186","1280x720")
            img_name = img_url.split("/")[-1]
            if "Slide01" in img_name and not img_name in imgs_set:
                print(img_url)
                name_file = "img_slide"+str(counter)
                img_data = requests.get(img_url).content
                with open(cwd+"/slides/"+name_file+".jpg", 'wb') as handler:
                    handler.write(img_data)
                    counter+=1
                imgs_set.add(img_name)
        except:
            continue

https://www.slideteam.net/media/catalog/product/cache/1280x720/0/5/0514_market_segmentation_powerpoint_presentation_Slide01.jpg
https://www.slideteam.net/media/catalog/product/cache/1280x720/m/a/market_segmentation_and_targeting_powerpoint_complete_deck_Slide01_1.jpg
https://www.slideteam.net/media/catalog/product/cache/1280x720/p/r/product_segmentation_markets_and_evaluation_powerpoint_presentation_slides_Slide01.jpg
https://www.slideteam.net/media/catalog/product/cache/1280x720/b/u/business_model_powerpoint_presentation_slides_Slide01.jpg
https://www.slideteam.net/media/catalog/product/cache/1280x720/m/a/market_segmentation_process_steps_powerpoint_presentation_slides_Slide01.jpg
https://www.slideteam.net/media/catalog/product/cache/1280x720/3/d/3d_market_segmentation_powerpoint_template_slide_Slide01.jpg
https://www.slideteam.net/media/catalog/product/cache/1280x720/m/a/marketing_segmentation_startegies_ppt_examples_slides_Slide01.jpg
https://www.slideteam.net/media/catalog/product/

In [59]:
try:
    from pytesseract import pytesseract 
except:
    %pip install pytesseract


try:
    from PIL import Image
except:
    %pip install pillow

#try:
#    import tesserocr
#except:
#    %pip install tesserocr
#    # in case of error with bd wheels or gcc exit with code 1
#    # sudo apt-get install gcc
#    # sudo apt install build-essential

# may need %pip install opencv-python-headless if opencv-python does not show images
import cv2
import os

# %pip install nltk
import nltk
# nltk.download('words')
from nltk.corpus import words as words
from nltk.corpus import wordnet


def meaningless_letters():
    from string import printable
    char_set = set(list(printable))
    char_set.discard('a'); char_set.discard('e'); char_set.discard('i'); char_set.discard('o'); char_set.discard('u')
    char_set.discard('0');char_set.discard('1');char_set.discard('2');char_set.discard('3');char_set.discard('4');char_set.discard('5')
    char_set.discard('6');char_set.discard('7');char_set.discard('8');char_set.discard('9')
    char_set.discard('x');char_set.discard('y')
    return char_set

all_words = set(wordnet.words()).union(set(words.words()))
all_words.difference_update(meaningless_letters())


#import requests
#from bs4 import BeautifulSoup

#url = "https://en.wikipedia.org/wiki/List_of_file_formats"
#text = requests.get(url).text
##print(text)
#print(text[text.find("xaml")-30:text.find("xaml")+30])
#soup = BeautifulSoup(text, "html.parser")





# UNUSED
#%pip uninstall -y english-words
#from english_words import english_words_set as words

path_to_images = os.path.join(os.getcwd(),"slides")
#print(path_to_images)
import random
img_file1 = "img_slide107.jpg"
for img_file in os.listdir(path_to_images):
    if random.random() < 0.2:
        img = Image.open(path_to_images+"/"+img_file1)
        print(img_file1)
        break
img_name = img_file1
import io

#with tesserocr.PyTessBaseAPI() as api:
#    image = img
#    api.SetImage(image)
#    api.Recognize()  # required to get result from the next line
#    iterator = api.GetIterator()
#    print(iterator.WordFontAttributes())

pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

img = cv2.cvtColor(cv2.imread(path_to_images+"/"+img_name), cv2.COLOR_BGR2RGB)


data = pytesseract.image_to_data(img, output_type='dict')

boxes = len(data['level'])


from nltk.stem.wordnet import WordNetLemmatizer
wnl = WordNetLemmatizer()
recognized_best_words = {"word":[],"height":[]}


for i in range(boxes):
    if data['text'][i].strip():
        lemma = wnl.lemmatize(data['text'][i].strip().lower())
        
    if data['text'][i].strip() and lemma in all_words:
        recognized_best_words['word'].append(lemma)
        (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        recognized_best_words['height'].append(h)
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #cv2.imshow("Image", final_img)
Image.fromarray(img).show()
print(recognized_best_words)

img_slide107.jpg
{'word': ['big', 'data', 'icon', 'in', 'gear', 'showing', 'connected', 'world', 'text', 'here', 'text', 'here', 'text', 'here', 'text', 'here', 'i', 'text', 'here', 'dale', 'yay', 'text', 'here', 'here', 'text', 'here', 'text', 'here', 'text', 'here', 'text', 'here', 'tor', 'here', 'text', 'here', 'your', 'logo'], 'height': [33, 25, 26, 26, 26, 35, 27, 27, 18, 18, 18, 18, 18, 18, 18, 18, 77, 18, 18, 163, 117, 18, 18, 32, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 12, 15]}
